# Pacotes

In [1]:
# Bibliotecas padrão e de manipulação de data
import pandas as pd
import numpy as np 
from unidecode import unidecode
import unidecode as uni
from pytz import timezone
import re

pd.set_option('display.max_columns', None)

# Visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt

# Machine Learning - pré-processamento e métricas
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold, KFold, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score, recall_score, precision_score, log_loss, roc_curve, average_precision_score, classification_report
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, precision_recall_curve, auc

# Machine Learning - Modelo de Regressão Logística
import statsmodels.api as sm

# Machine Learning - modelos e otimização
import xgboost as xgb
from xgboost import plot_importance
from xgboost import plot_tree
import sklearn
import shap
from shap import Explainer
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK, space_eval,SparkTrials
from hyperopt.pyll import scope
from skopt import forest_minimize
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline

# Salvamento e carregamento de modelos
import pickle
import os
import mlflow

# Outras bibliotecas

from mlflow.models import infer_signature
from mlflow.models.signature import infer_signature

import warnings

warnings.filterwarnings("ignore")

plt.style.use('ggplot')
print(xgb.__version__)
print(shap.__version__)
print(sklearn.__version__)
#print(hyperopt.__version__)

2.0.2
0.44.0
1.2.0


# Funções

In [2]:
def tira_acentos(nome):
    """Remove acentos de strings."""
    txt = uni.unidecode(str(nome))
    return txt

def corrigir_nomes(nome):
    """Remove pontos de strings."""
    nome = nome.replace('.', '')
    return nome

## Dummies
def get_one_hot_enc(feature_col: pd.Series, enc: preprocessing.OneHotEncoder, column_name: str) -> pd.DataFrame:
    """
    Aplica codificação one-hot a uma coluna específica de um DataFrame.

    Args:
    feature_col (pd.Series): Coluna do DataFrame para ser codificada.
    enc (preprocessing.OneHotEncoder): Instância do OneHotEncoder treinada.
    column_name (str): Nome da coluna a ser codificada.

    Processo:
    1. Cria um DataFrame com a coluna especificada.
    2. Aplica a transformação one-hot usando o encoder fornecido.
    3. Retorna um DataFrame com as colunas codificadas.

    Returns:
    pd.DataFrame: DataFrame com as colunas resultantes da codificação one-hot.
    """
    unseen_df = pd.DataFrame(feature_col, columns=[column_name])
    encoded_vec = enc.transform(unseen_df).toarray()
    columns = enc.get_feature_names_out([column_name])
    encoded_df = pd.DataFrame(encoded_vec, columns=columns)
    return encoded_df


# Função para salvar um modelo em um arquivo pickle
def save_pickle(model, file_name: str):
    """
    Salva um objeto modelo em um arquivo pickle.

    Args:
    model (any): O modelo a ser salvo.
    file_name (str): Nome do arquivo pickle a ser criado.

    Processo:
    1. Abre o arquivo pickle em modo de escrita binária.
    2. Salva o objeto modelo no arquivo pickle.
    """
    with open(file_name, 'wb') as file:
        pickle.dump(model, file)

# Função para carregar um modelo de um arquivo pickle
def load_pickle(file_name: str) -> any:
    """
    Carrega um objeto a partir de um arquivo pickle.

    Args:
    file_name (str): Nome do arquivo pickle a ser carregado.

    Processo:
    1. Abre o arquivo pickle em modo de leitura binária.
    2. Carrega e retorna o objeto contido no arquivo pickle.

    Returns:
    any: Objeto carregado do arquivo pickle.
    """
    with open(file_name, 'rb') as file:
        return pickle.load(file)

def print_class_distribution(y, label):
    unique, counts = np.unique(y, return_counts=True)
    distribution = dict(zip(unique, counts))
    total = sum(counts)
    print(f'Distribuição de classes {label}:')
    for k, v in distribution.items():
        print(f'Classe {k}: {v/total:.2%}')
    print('--------------------')

def log_class_distribution(y, label):
    unique, counts = np.unique(y, return_counts=True)
    distribution = dict(zip(unique, counts))
    total = sum(counts)
    print(f'Distribuição de classes {label}:')
    for k, v in distribution.items():
        percentage = v / total
        print(f'Classe {k}: {percentage:.2%}')
        # Logar no MLflow
        mlflow.log_metric(f"{label}_classe_{k}_prop", percentage)
    print('--------------------')


# Função para limpar os nomes dos parâmetros
def clean_param_name(name):
    # Substitui vírgulas e remove acentuação e caracteres especiais
    name = re.sub(r'[^\w\s.-]', '', name)
    name = re.sub(r'\s+', '_', name)  # Substitui espaços por sublinhados
    return name

def unified_hyper_tuning(space):
    """
    Realiza o ajuste de hiperparâmetros e treinamento de um modelo XGBoost com logging completo utilizando MLflow.
    
    Args:
        space (dict): Dicionário contendo os hiperparâmetros para o modelo XGBoost.
        
    Returns:
        dict: Dicionário contendo o 'loss' (negativo da média do AUC) e o 'status'.
    """
    mlflow.xgboost.autolog()
    with mlflow.start_run(experiment_id=experiment_id, run_name='Unified Model Training and Tuning', nested=True):
        #  Configuração do modelo com os parâmetros do espaço
        clf = xgb.XGBClassifier(max_depth = space['max_depth'],
                                  learning_rate = space['learning_rate'],
                                  reg_alpha = space['reg_alpha'],
                                  reg_lambda = space['reg_lambda'],
                                  min_child_weight = space['min_child_weight'],
                                  subsample = space['subsample'],
                                  colsample_bytree = space['colsample_bytree'],
                                  gamma = space['gamma'],
                                  objective = space['objective'],
                                  seed = space['seed'])
        
        # StratifiedKFold para manter a proporção de classes em cada fold
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        # Avaliação usando cross_val_score no conjunto de treinamento
        auc_scores = cross_val_score(clf, X_train, y_train, cv=skf, scoring='roc_auc')
        mean_auc = auc_scores.mean()

        # Logando a média do AUC
        mlflow.log_metric('mean_auc', mean_auc)
        
        model = clf.fit(X_smote_a, y_smote_a)
        
        
        # Teste do modelo
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    
        # Teste do modelo e log das curvas
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    
        # Plotar e salvar a Curva de Precisão-Recall
        plt.figure(figsize=(8, 6))
        plt.plot(recall, precision, marker='.')
        plt.title('Curva de Precisão-Recall')
        plt.xlabel('Recall')
        plt.ylabel('Precisão')
        plt.savefig('precision_recall_curve.png')
        plt.close()
    
        # Plotar e salvar a Curva ROC
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, linestyle='--')
        plt.title('Curva ROC')
        plt.xlabel('Taxa de Falso Positivo')
        plt.ylabel('Taxa de Verdadeiro Positivo')
        plt.savefig('roc_curve.png')
        plt.close()
    
        # Logar gráficos como artefatos
        mlflow.log_artifact('precision_recall_curve.png')
        mlflow.log_artifact('roc_curve.png')

        # Create a model signature
        signature = infer_signature(X_test, model.predict(X_test))
        model_info = mlflow.xgboost.log_model(model, "modelo_xgboost", signature=signature) 
        
        mlflow.xgboost.log_model(model, "model_xgb", signature=signature)
        model_uri = mlflow.get_artifact_uri("model_xgb")
        
        eval_data = pd.DataFrame(X_test, columns=dt_ax.columns)
        eval_data['atraso30_m3'] = y_test.reset_index(drop=True)
        
        result = mlflow.evaluate(model_uri,
                                 eval_data,
                                 targets="atraso30_m3",
                                 model_type="classifier",
                                 evaluators=["default"])

        # A função de perda é o negativo da média do AUC para otimização
        return {'loss': -mean_auc, 'status': STATUS_OK}

space = {
  'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
  'learning_rate': hp.loguniform('learning_rate', -3, 0),
  'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
  'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
  'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
  'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
  'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
  'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
  'objective': 'binary:logistic',
  'seed': 123, # Set a seed for deterministic training
}

def clean_column_names(column_names):
    """Cria um dicionário com os nomes das colunas antigos e os tratados para renomeação.

    Args:
        column_names (list of str): Lista com os nomes das colunas originais.

    Returns:
        dict: Dicionário com os nomes antigos como chaves e os tratados como valores.
    """
    name_map = {}
    for name in column_names:
        # Substitui espaços e pontos por underline
        clean_name = re.sub(r'[\s\.]', '_', name)
        
        # Substitui caracteres especiais por vazio
        clean_name = re.sub(r'[^\w<>]', '', clean_name)
        
        # Substitui '>' e '<' por 'maior' e 'menor'
        clean_name = clean_name.replace('>', 'maior').replace('<', 'menor')
        
        # Assegura que não haja múltiplos underlines consecutivos
        clean_name = re.sub(r'_{2,}', '_', clean_name)
        
        # Remove possíveis underlines no início ou fim do nome
        clean_name = clean_name.strip('_')
        
        name_map[name] = clean_name
    return name_map
    

# dataset

# Definindo um experimento

In [3]:
# Definindo um experimento
mlflow.set_experiment("Case aviação area brasileira")
# local. Importante direcionar a localização do arquivo de experimento.
# mlflow ui --backend-store-uri file:///C:/Users/pedro/Desktop/estudo%20mlflow/mlruns

2024/04/27 15:02:59 INFO mlflow.tracking.fluent: Experiment with name 'Case aviação area brasileira' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/pedro/Desktop/estudo%20mlflow/mlruns/452140177683560545', creation_time=1714240979122, experiment_id='452140177683560545', last_update_time=1714240979122, lifecycle_stage='active', name='Case aviação area brasileira', tags={}>

In [4]:
# Substitua 'Nome do Experimento' pelo nome que deseja verificar ou criar
experiment_name = "Case aviação area brasileira"
experiment = mlflow.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id

In [5]:
experiment_id

'452140177683560545'

# Tratamento de dados

In [7]:
# Start an MLflow run context
with mlflow.start_run(experiment_id=experiment_id, run_name='extração e tratamento dos dados', 
                      run_id='35590baf1ae1467399becbb577f67862',
                      description = 'Extração e/ou tratamento de dados',
                      tags = {"Extração": "origem_x", "objetivo": "alimentar o modelo_x", "Versão da etapa": "1.0"}):
    file_path = 'historico_voo_tratados_train.csv'
    df = pd.read_csv(file_path)

# Treinamento

## Pre-processamento

### One hot encoding

In [28]:
with mlflow.start_run(experiment_id=experiment_id, run_name='Pre-processamento',
                      run_id='ba596ce0c0ff43228f85f3ef932a8310',
                      nested=True,
                      description = 'Garantir o input correto dos modelos',
                      tags = {"Pre-processamento": "preparação para treinamento", "objetivo": "garantir o input correto dos dados", "Versão da etapa": "1.0"}):
    
    chosen_columns = ['nome_empresas','codigo_tipo_linha','descricao_origem','descricao_destino','pais_origem','pais_destino','continente_origem',
                 'continente_destino','cidade_origem','cidade_destino','uf_origem','uf_destino','mes_partida',
                 'dia_semana_chegada','status_do_voo']
    
    dft = df[chosen_columns].sample(frac=0.3, random_state=13)
    
    # Colunas que precisam passar por one hot encoding
    list_dummies = ['nome_empresas','codigo_tipo_linha','descricao_origem','descricao_destino','pais_origem','pais_destino','continente_origem',
                 'continente_destino','cidade_origem','cidade_destino','uf_origem','uf_destino','mes_partida',
                 'dia_semana_chegada']

    final_data = pd.DataFrame()
    # Logar os parâmetros
    mlflow.log_param("Colunas escolhidas", chosen_columns)
    mlflow.log_param("Index", 'num_cpf')
    mlflow.log_param("Colunas para one-hot encoding", list_dummies)
    
    # Logar métricas
    mlflow.log_metric("Quantidade de colunas", len(chosen_columns))
    mlflow.log_metric("Quantidade de colunas dummies", len(list_dummies))
    mlflow.log_metric("Quantidade de colunas não dummies", len(chosen_columns) - len(list_dummies) - 1) 
    
    ### One hot encoding
    with mlflow.start_run(experiment_id=experiment_id, nested=True, run_name='One hot encoding', run_id='8836439277bc460e8767f9e6b7311883',
                      description = 'Transformação das colunas categoricas em númericas',
                      tags = {"One hot encoding": "Transformar categorica em númerica", "objetivo": "garantir o input correto dos dados", "Versão da etapa": "1.0"}):
        for column in list_dummies:
            encoder = preprocessing.OneHotEncoder(handle_unknown='ignore')
            encoder.fit(dft[[column]])
            
            # Logar parâmetros para cada coluna processada
            mlflow.log_param(f"Coluna_{column.lower()}", column.lower())
            
            enc_df = pd.DataFrame(encoder.transform(dft[[column]]).toarray(), 
                                  columns=encoder.get_feature_names_out([column]))
            final_data = pd.concat([final_data, enc_df], axis=1)

        final_data['status_do_voo'] = dft['status_do_voo'].values

        dt_ax = final_data.drop(columns=["status_do_voo"])
        dt_ay = final_data[['status_do_voo']].copy()

        # Transformação da coluna em valores binarios. Pontual = 1 e Atrasado = 0
        label_encoder = LabelEncoder()
        dt_ay_enc = label_encoder.fit_transform(dt_ay)
        dt_ay_df = pd.DataFrame(dt_ay_enc, columns=dt_ay.columns)

        # Suponha que 'df' é o seu DataFrame
        column_names = dt_ax.columns.tolist()
        name_map = clean_column_names(column_names)
        
        # Renomear colunas no DataFrame
        dt_ax.rename(columns=name_map, inplace=True)
        
    ### Normalização / Segmentação  treino e teste / Smote
    with mlflow.start_run(experiment_id=experiment_id, run_name='Normalização e Smote', nested=True,run_id='a058314be1ff4283b8fafd1168611eba',
                      description = 'Implementação da etapa de normalização e SMOTE dos dados. Essas etapas são essenciais para evitar overfiting e underfitting',
                      tags = {"Normalização e SMOTE": "Normalização em range de 0 a 1 e criação de dados sinteticos para balencear", "objetivo": "garantir qualidade no correto dos dados", "Versão da etapa": "1.0"}):
        # Normalização dos dados
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(dt_ax)
        X_scaled_df = pd.DataFrame(X_scaled, columns=dt_ax.columns)
    
        # Segmentação em Treino (85%) e Teste (15%)
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, dt_ay_df, random_state=13, test_size=0.15)
    
        # Logar distribuição das classes antes do SMOTE
        log_class_distribution(y_test, 'original')
    
        # Aplicar SMOTE
        smote = SMOTE(random_state=13)
        X_smote_a, y_smote_a = smote.fit_resample(X_train, y_train)

        X_test = X_test.reset_index().drop(columns = 'index')
        y_test = y_test.reset_index().drop(columns = 'index')
    
        # Logar distribuição das classes após SMOTE
        log_class_distribution(y_smote_a, 'SMOTE')

Distribuição de classes original:
Classe 0: 13.62%
Classe 1: 86.38%
--------------------
Distribuição de classes SMOTE:
Classe 0: 50.00%
Classe 1: 50.00%
--------------------


## XGBoost

### Hipertunnig

In [34]:
# Etapa de hipertunning
with mlflow.start_run(experiment_id=experiment_id, run_name='Hipertunnig', nested=True,run_id='a51480d6bb1f41cf99cee7b9d87e7dc0',
                      description = 'Busca pelos melhores parametros. Os modelos testados são armazenados, mesmo que não tenha os melhores parametros.',
                      tags = {"Hipertunnig": "Melhores parametros", "objetivo": "garantir os melhores parametros para o modelo", "Versão da etapa": "1.0"}):
    # Executando a otimização
    trials = Trials()
    best_hyperparams = fmin(fn=unified_hyper_tuning, 
                            space=space, 
                            algo=tpe.suggest, 
                            max_evals=5, 
                            trials=trials)
    
    # Obtendo os melhores hiperparâmetros
    mlflow.log_params(best_hyperparams)
    best_hyperparams = space_eval(space, best_hyperparams)
    print("Melhores hiperparâmetros:", best_hyperparams)

  0%|                                                                            | 0/5 [00:00<?, ?trial/s, best loss=?]

2024/04/27 15:49:07 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.

2024/04/27 15:49:07 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.

2024/04/27 15:49:08 INFO mlflow.models.evaluation.default_evaluator: The evaluation dataset is inferred as binary dataset, positive label is 1, negative label is 0.

2024/04/27 15:49:08 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...

2024/04/27 15:49:10 INFO mlflow.models.evaluation.default_evaluator: Shap explainer PermutationExplainer is used.

PermutationExplainer explainer:   1%|6                                                       | 23/2000 [00:00<?, ?it/s]

PermutationExplainer explainer:   1%|6                                               | 25/2000 [00:10<11:10,  2.94it/s]

PermutationExplainer explainer:   1%|6                                               | 26/2000 [00:11<09:53,  3.32it/s]

PermutationExplainer explainer:   1%|6           

 20%|█████████▏                                    | 1/5 [15:06<1:00:26, 906.71s/trial, best loss: -0.6491167693557198]

2024/04/27 16:04:36 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.

2024/04/27 16:04:36 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.

2024/04/27 16:04:37 INFO mlflow.models.evaluation.default_evaluator: The evaluation dataset is inferred as binary dataset, positive label is 1, negative label is 0.

2024/04/27 16:04:37 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...

2024/04/27 16:04:39 INFO mlflow.models.evaluation.default_evaluator: Shap explainer PermutationExplainer is used.

PermutationExplainer explainer:   2%|#                                                       | 38/2000 [00:00<?, ?it/s]

PermutationExplainer explainer:   2%|9                                               | 40/2000 [00:10<04:31,  7.22it/s]

PermutationExplainer explainer:   2%|9                                               | 41/2000 [00:10<06:04,  5.37it/s]

PermutationExplainer explainer:   2%|#           

 40%|███████████████████▏                            | 2/5 [30:39<46:05, 921.97s/trial, best loss: -0.6491167693557198]

...

 60%|█████████████████████████████▍                   | 3/5 [46:02<30:45, 922.70s/trial, best loss: -0.660514220289836]

2024/04/27 16:35:12 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.

2024/04/27 16:35:12 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.

2024/04/27 16:35:12 INFO mlflow.models.evaluation.default_evaluator: The evaluation dataset is inferred as binary dataset, positive label is 1, negative label is 0.

2024/04/27 16:35:12 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...

2024/04/27 16:35:15 INFO mlflow.models.evaluation.default_evaluator: Shap explainer PermutationExplainer is used.

PermutationExplainer explainer:   2%|#1                                                      | 41/2000 [00:00<?, ?it/s]

PermutationExplainer explainer:   2%|#                                               | 43/2000 [00:10<03:52,  8.41it/s]

PermutationExplainer explainer:   2%|#                                               | 44/2000 [00:10<05:35,  5.83it/s]

PermutationExplainer explainer:   2%|#           

 80%|█████████████████████████████████████▌         | 4/5 [1:01:05<15:14, 914.57s/trial, best loss: -0.660514220289836]

2024/04/27 16:50:13 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.

2024/04/27 16:50:13 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.

2024/04/27 16:50:13 INFO mlflow.models.evaluation.default_evaluator: The evaluation dataset is inferred as binary dataset, positive label is 1, negative label is 0.

2024/04/27 16:50:13 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...

2024/04/27 16:50:15 INFO mlflow.models.evaluation.default_evaluator: Shap explainer PermutationExplainer is used.

PermutationExplainer explainer:   2%|#1                                                      | 41/2000 [00:00<?, ?it/s]

PermutationExplainer explainer:   2%|#                                               | 43/2000 [00:11<22:03,  1.48it/s]

PermutationExplainer explainer:   2%|#                                               | 44/2000 [00:11<16:11,  2.01it/s]

PermutationExplainer explainer:   2%|#           

100%|██████████████████████████████████████████████| 5/5 [1:16:05<00:00, 913.12s/trial, best loss: -0.6625767202830284]
Melhores hiperparâmetros: {'colsample_bytree': 0.75, 'gamma': 0.7000000000000001, 'learning_rate': 0.11946628062413302, 'max_depth': 22, 'min_child_weight': 3.0, 'objective': 'binary:logistic', 'reg_alpha': 0.036495399966362974, 'reg_lambda': 0.006691827695655175, 'seed': 123, 'subsample': 0.8}


In [35]:
best_hyperparams = space_eval(space, best_hyperparams)
best_hyperparams

{'colsample_bytree': 0.75,
 'gamma': 0.7000000000000001,
 'learning_rate': 0.11946628062413302,
 'max_depth': 22,
 'min_child_weight': 3.0,
 'objective': 'binary:logistic',
 'reg_alpha': 0.036495399966362974,
 'reg_lambda': 0.006691827695655175,
 'seed': 123,
 'subsample': 0.8}

### Treinamento do Modelo XGBoost com melhores parametros

In [36]:
# Iniciar rastreamento MLflow
with mlflow.start_run(experiment_id=experiment_id, run_name='Treinamento do melhor modelo modelo XGBoost', nested=True, run_id = 'b146f1b10b7c4d1b94ec262f6dffb274',
                     description = 'Treinando o XGBoost com os melhores parametros',
                     tags = {"Versão do modelo": "1", "Algoritmo": "Xgboost"}):
    mlflow.log_params(best_hyperparams)

    # Treinamento do Modelo XGBoost com melhores parâmetros
    model = xgb.XGBClassifier(max_depth = space['max_depth'],
                                  learning_rate = space['learning_rate'],
                                  reg_alpha = space['reg_alpha'],
                                  reg_lambda = space['reg_lambda'],
                                  min_child_weight = space['min_child_weight'],
                                  subsample = space['subsample'],
                                  colsample_bytree = space['colsample_bytree'],
                                  gamma = space['gamma'],
                                  objective = space['objective'],
                                  seed = space['seed'])

    # Treino do modelo utilizando dados do SMOTE 
    model.fit(X_smote_a, y_smote_a)

    # Logar o modelo
    mlflow.xgboost.log_model(model, "model")

    # Teste do modelo
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    # Teste do modelo e log das curvas
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)

    # Plotar e salvar a Curva de Precisão-Recall
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, marker='.')
    plt.title('Curva de Precisão-Recall')
    plt.xlabel('Recall')
    plt.ylabel('Precisão')
    plt.savefig('precision_recall_curve.png')
    plt.close()

    # Plotar e salvar a Curva ROC
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, linestyle='--')
    plt.title('Curva ROC')
    plt.xlabel('Taxa de Falso Positivo')
    plt.ylabel('Taxa de Verdadeiro Positivo')
    plt.savefig('roc_curve.png')
    plt.close()

    # Logar gráficos como artefatos
    mlflow.log_artifact('precision_recall_curve.png')
    mlflow.log_artifact('roc_curve.png')

    # Create a model signature
    signature = infer_signature(X_test, model.predict(X_test))
    mlflow.xgboost.log_model(model, "model_xgb", signature=signature)
    model_uri = mlflow.get_artifact_uri("model_xgb")
    
    eval_data = pd.DataFrame(X_test, columns=dt_ax.columns)
    eval_data['atraso30_m3'] = y_test.reset_index(drop=True)

    # Evaluate the logged model
    result = mlflow.evaluate(
        model_uri,
        eval_data,
        targets="atraso30_m3",
        model_type="classifier",
        evaluators=["default"])

2024/04/27 17:02:06 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/04/27 17:02:06 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/04/27 17:02:07 INFO mlflow.models.evaluation.default_evaluator: The evaluation dataset is inferred as binary dataset, positive label is 1, negative label is 0.
2024/04/27 17:02:07 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/04/27 17:02:09 INFO mlflow.models.evaluation.default_evaluator: Shap explainer PermutationExplainer is used.
PermutationExplainer explainer: 2001it [08:57,  3.66it/s]                                                              


## Statsmodels

Obs.: O resultado do modelo abaixo não ficou bom. O objetivo aqui é utilizar o statmodels como exemplo prático em cima de um conjuto de dados. 

In [51]:
with mlflow.start_run(experiment_id=experiment_id, run_name='Treinamento de regressão com Statsmodels', nested=True, run_id = '496bc7ea97784b01bacdf67f57df5cb0', 
                     description = 'Teste implementando statmodels no mlflow',
                     tags = {"teste_statmodels": "deploy statmodel moflow", "objetivo": "testar biblioteca statmodels+mlfow", "Versão do teste": "1.0"}):
    mlflow.statsmodels.autolog()
    
    if 'const' not in X_smote_a.columns:
        X_train = sm.add_constant(X_smote_a, has_constant='add')
    else:
        X_train = X_smote_a

    if 'const' not in X_test.columns:
        X_test = sm.add_constant(X_test, has_constant='add')
    else:
        X_test = X_test

    # Criar e ajustar o modelo
    model = sm.Logit(y_smote_a, X_train)
    result = model.fit(method='bfgs')

    cleaned_params = {clean_param_name(k): v for k, v in dict(result.params).items()}
    mlflow.log_params(cleaned_params)

    # Fazer previsões
    y_pred_proba = result.predict(X_test)
    y_pred = (y_pred_proba > 0.5).astype(int)

    # Teste do modelo e log das curvas
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)

    # Avaliar e logar métricas
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("AUC ROC", roc_auc)
    mlflow.log_metric("Accuracy", accuracy)

    # Matriz de Confusão
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6, 4))
    sns.heatmap(conf_matrix, annot=True, fmt='d')
    plt.title('Matriz de Confusão')
    conf_matrix_path = 'confusion_matrix.png'
    plt.savefig(conf_matrix_path)
    plt.close()
    mlflow.log_artifact(conf_matrix_path)

    # Coeficientes do Modelo
    plt.figure(figsize=(8, 6))
    result.params[1:].sort_values().plot(kind='bar')
    plt.title('Coeficientes do Modelo de Regressão Logística')
    coefficients_path = 'model_coefficients.png'
    plt.savefig(coefficients_path)
    plt.close()
    mlflow.log_artifact(coefficients_path)

    # Logar gráficos de desempenho do modelo
    plt.figure(figsize=(8, 6))
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    plt.plot(recall, precision, marker='.')
    plt.title('Curva de Precisão-Recall')
    plt.xlabel('Recall')
    plt.ylabel('Precisão')
    pr_curve_path = 'precision_recall_curve.png'
    plt.savefig(pr_curve_path)
    plt.close()
    mlflow.log_artifact(pr_curve_path)

    plt.figure(figsize=(8, 6))
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    plt.plot(fpr, tpr, linestyle='--')
    plt.title('Curva ROC')
    plt.xlabel('Taxa de Falso Positivo')
    plt.ylabel('Taxa de Verdadeiro Positivo')
    roc_curve_path = 'roc_curve.png'
    plt.savefig(roc_curve_path)
    plt.close()
    mlflow.log_artifact(roc_curve_path)

    # Logando o modelo
    mlflow.statsmodels.log_model(result, "model_statsmodels")

    report_dict = classification_report(y_test, y_pred, output_dict=True)
    # Logar métricas detalhadas no MLflow
    for label, metrics in report_dict.items():
        if label not in ["accuracy", "macro avg", "weighted avg"]:  # Ignorando a média total por enquanto
            mlflow.log_metric(f"Precision Classe {label}", metrics['precision'])
            mlflow.log_metric(f"Recall Classe {label}", metrics['recall'])
            mlflow.log_metric(f"F1-score Classe {label}", metrics['f1-score'])
            mlflow.log_metric(f"Support Classe {label}", metrics['support'])

    # Métricas globais
    mlflow.log_metric("Accuracy Total", report_dict['accuracy'])
    mlflow.log_metric("Macro Avg Precision", report_dict['macro avg']['precision'])
    mlflow.log_metric("Macro Avg Recall", report_dict['macro avg']['recall'])
    mlflow.log_metric("Macro Avg F1-score", report_dict['macro avg']['f1-score'])
    mlflow.log_metric("Weighted Avg Precision", report_dict['weighted avg']['precision'])
    mlflow.log_metric("Weighted Avg Recall", report_dict['weighted avg']['recall'])
    mlflow.log_metric("Weighted Avg F1-score", report_dict['weighted avg']['f1-score'])

    # Exibir resultados adicionais
    print("AUC-ROC:", roc_auc)
    print("Acurácia:", accuracy)
    print(classification_report(y_test, y_pred))
    
    # Avaliação com MLflow (Opcional se MLflow suporta evaluate para statsmodels)
    eval_data = pd.DataFrame(X_test)
    eval_data['real'] = y_test.reset_index(drop=True)
    model_uri = mlflow.get_artifact_uri("model_statsmodels")

         Current function value: 0.649085
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36


2024/04/27 17:35:17 WARNING mlflow.statsmodels: The fitted model is larger than 100 MB, saving it as artifacts is time consuming.
To reduce model size, use `mlflow.statsmodels.autolog(log_models=False)` and manually log model by `mlflow.statsmodels.log_model(model, remove_data=True, artifact_path="model")`


AUC-ROC: 0.6367040967191571
Acurácia: 0.5986705328810592
              precision    recall  f1-score   support

           0       0.19      0.59      0.28      4979
           1       0.90      0.60      0.72     31577

    accuracy                           0.60     36556
   macro avg       0.55      0.59      0.50     36556
weighted avg       0.80      0.60      0.66     36556



In [ ]:
mlflow.end_run()